# Creacion de Modelo con el maximo valor f1 posible.

## Introducción
Utilizaremos el conjunto de datos `/datasets/Churn.csv` para crear un modelo con el maximo valor f1 posible, tambien mediremos la métrica AUC-ROC y la compararemos con el valor F1. Para esto llevaremos a cabo las siguientes etapas.

### Etapas
1. Importaremos las librerias y prepararemos los datos.
   * 1.1 Importaremos las librerias.
   * 1.2 Leeremos y prepararemos los datos.
2. Procesar todos los tipos de caracteristicas
3. Examinaremos el equilibrio de clases y entrenaremos el modelo sin tener en cuenta el desequilibrio.
    * 2.2 Entrenar modelos sin tomar en cuenta el desequilibrio
    * 2.3 Examinar el equilibrio
4. Mejoraremos la Calidad del modelo.
    * 4.1 Sobremuestreo
    * 4.2 Submuestreo
5. Examinaremos los valores AUC-ROC
6. Realizaremos la prueba final.
7. Escribiremos un conclusion general.

## Importaremos las librerias

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

## Leeremos y prepararemos los datos

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.info()
data = data.dropna(subset=['Tenure'])
print('\nNumero de valores duplicados:',data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB

Numero de valores duplicados: 0


In [4]:
#Eliminacion de columnas inecesarias
data_s = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
display(data_s.head())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


**OBSERVACIONES**
Nuestro conjunto de datos esta listo para poder llevar a cabo la creación de modelos ya que no tenemos valores faltantes ni duplicados. Al principio el conjunto de datos tenia datos faltantes en la columna `Tenure` pero debido a que eran una cantidad muy pequeña obtamos por eliminarlos. Tambien eliminaremos las columnas `RowNumber, Customerid y Surname` ya que son irrelevantes para nuestro modelo y solo harian el proceso mas lento ya que Surname al ser una columna categorica si usaramos OHE con esta columna crearia muchas columnas inecesarias.

## Procesar todos los tipos de caracteristicas

En nuestro conjunto de datos tenemos columnas con datos categoricos que necesitan ser cambiadas a variables binarias, para lograrlo usaremos OHE(One-Hot-Encoding) en todo el conjunto de datos. 

In [5]:
# Convertimos los valores categoricos a binarios
data_ohe = pd.get_dummies(data_s, drop_first=True)
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Ahora dividiremos nuestro conjunto de datos en 3 partes, El primer conjunto sera el de entrenamiento con un 60 porciente, el segundo conjunto sera para el de validacion con 20 y por ultimo sera el conjunto de prueba con el 20 porciento faltante.

In [6]:
# Primero dividimos nuestro conjunto de datos en 60% para el entrenamiento, 20 para el conjunto de validacion
# 20 para el conjunto de prueba.

features = data_ohe.drop(['Exited'], axis = 1)
target = data_ohe['Exited']

features_train, features_valid, target_train, target_valid = train_test_split(features, 
target, test_size= .4 , random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
target_valid, test_size= .50 , random_state=12345)

Ahora estandarizaremos las características numéricas con StandardScaler.

In [7]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train  = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
features_test = scaler.transform(features_test)

## Examinaremos el equilibrio de clases y entrenaremos el modelo sin tener en cuenta el desequilibrio.   

### Entrenaremos los modelos sin tomar en cuenta el desequilibrio

In [8]:
# Modelo Arbol de Decision
best_model_td = None
best_result = 0 
best_depth = 0
for depth in range(1, 6): # seleccionaremos el rango del hiperparámetro
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
    model.fit(features_train, target_train) # entrenaremos el modelo con el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid) 
    result =  f1_score(target_valid, predictions_valid)
    if result > best_result:
        best_model_td = model
        best_result = result
        best_depth = depth

print(f"f1 del mejor modelo con el conjunto de validación (max_depth = {best_depth}): {best_result}")

f1 del mejor modelo con el conjunto de validación (max_depth = 4): 0.5414551607445008


In [9]:
# Crearemos nuestro Modelo Bosque aleatorio de regresion

best_model_rf = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # entrenaremos el modelo con el conjunto de entrenamiento
        predictions_valid = model.predict(features_valid) # obténdremos las predicciones del modelo con el conjunto de validación
        result = f1_score(target_valid, predictions_valid) 
        if result > best_result:
            best_model_rf = model
            best_result = result
            best_est = est
            best_depth = depth

print("f1 del mejor modelo con el conjunto de validación", best_result, "n_estimators:", best_est, "best_depth:", depth)

f1 del mejor modelo con el conjunto de validación 0.5758620689655172 n_estimators: 20 best_depth: 9


In [10]:
# Modelo Regresion Logistica

model = LogisticRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
result = f1_score(target_valid, predicted_valid)
print(result)

0.3033932135728543


Ninguno de nuestros modelos logra alcanzar el objetivo de tener un calificacion de f1 minima de .59, nuestro modelo mas cercano fue el del `bosque aleatorio` con un `.57` despues el `arbol de decicion` con `.54` y en ultimo lugar el modelo de Regresion Logistica con `0.30`. En la siguiente seccion haremos las modificaciones necesarias para obtener una puntuacion minima de 0.59, muy probablemente el modelo de bosque aleatorio tendra un mejor resultado con nuestras modificaciones.

### Examinacion del equilibrio de clases

In [11]:
 equilibrio = classification_report(target_valid, predicted_valid)
print(equilibrio)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89      1441
           1       0.61      0.20      0.30       377

    accuracy                           0.81      1818
   macro avg       0.72      0.58      0.60      1818
weighted avg       0.78      0.81      0.77      1818



Nuestro modelo es bueno para predecir zeros pero es malo para predecir unos y eso tiene sentido, en la columna support podemos ver que tenemos muchos mas datos para los zeros que para los unos, asi que podemos ver que tenemos una buen precision y recall en los zeros por lo tanto tenemos una buena calificacion f1 para los zeros, pero debido a que en los unos tenemos menos datos, eso hace que tengamos una calificacion baja en la precision, en el recall y en la calificacion f1. Para solucionar esto aumentaremos los unos para que tengan una proporcion similar a los zeros y ayude a que mejore la calificacion. Lo que significaria que nuestro modelo seria mejor para predecir unos que es donde le hace falta mejorar.

# Mejoraremos la Calidad del Modelo 

### Iniciaremos con un sobremuestreo para cada modelo con las clases balanceadas

In [12]:
# Aplicar sobremuestreo

def upsample(features, target, repeat):
    features_zeros = pd.DataFrame(features[target == 0])
    features_ones = pd.DataFrame(features[target == 1])
    target_zeros = pd.Series(target[target == 0])
    target_ones = pd.Series(target[target == 1])
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled


features_upsampled, target_upsampled = upsample(
    features_train, target_train, 5
)
print(target_upsampled.value_counts())

model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print('\nF1:', f1_score(target_valid, predicted_valid))

1    5630
0    4328
Name: Exited, dtype: int64

F1: 0.4859211584875302


In [13]:
# Modelo Arbol de Decision
best_model_sob = None
best_result = 0 
best_depth = 0
for depth in range(1, 6): # seleccionaremos el rango del hiperparámetro
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345, class_weight = 'balanced')
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid) 
    result =  f1_score(target_valid, predictions_valid)
    if result > best_result:
        best_model_sob = model
        best_result = result
        best_depth = depth

print(f"f1 con sobremuestreo del mejor modelo con el conjunto de validación (max_depth = {best_depth}): {best_result}")


f1 con sobremuestreo del mejor modelo con el conjunto de validación (max_depth = 5): 0.5735449735449736


In [14]:
best_model_rf_sob = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range (1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight = 'balanced')
        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid) # obténdremos las predicciones del modelo con el conjunto de validación
        result = f1_score(target_valid, predictions_valid) 
        if result > best_result:
            best_model_rf_sob = model
            best_result = result
            best_est = est
            best_depth = depth

print("f1 con sobremuestreo del mejor modelo con el conjunto de validación", best_result, "n_estimators:", best_est, "best_depth:", depth)

f1 con sobremuestreo del mejor modelo con el conjunto de validación 0.641291810841984 n_estimators: 100 best_depth: 9


Al realizar el sobremuestreo y aplicarlo en cada uno de nuestros modelos, vemos claramente que ha aumentado la calificacion f1 en todos nuestros modelos. Lo que hizimos fue ver la diferencia que existian entre los zeros y unos y aumentamos los unos ya que aparecian mucho menos que los zeros. Para poder saber por cual numero teniamos que multiplicar los unos solo imprimiamos `print(target_upsampled.value_counts())` y modificabamos el numero que nos aproximara mas la cantidad de zeros y unos. Hacer esto ayudo mucho al modelo a predecir mejor los unos ya que le damos una mayor cantidad de unos que el pueda usar para entrenarse. Podemos destacar que el modelo `best_model_rf_sob` obtuvo una calificacion de `0.64` siendo superior a nuestra meta de `0.59` por lo cual este es nuetro mejor modelo con el mejor conjunto de datos de entrenamiento.

### Ahora haremos un Submuestreo para cada modelo con las clases balanceadas

In [15]:
def downsample(features, target, fraction):
    features_zeros = pd.DataFrame(features[target == 0])
    features_ones = pd.DataFrame(features[target == 1])
    target_zeros = pd.Series(target[target == 0])
    target_ones = pd.Series(target[target == 1])

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)]
        + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)]
        + [target_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )

    return features_downsampled, target_downsampled


features_downsampled, target_downsampled = downsample(
    features_train, target_train, 0.30)
print(target_downsampled.value_counts())
model = LogisticRegression(random_state = 12345, solver = 'liblinear', class_weight = 'balanced')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print('\nF1:', f1_score(target_valid, predicted_valid))

0    1298
1    1126
Name: Exited, dtype: int64

F1: 0.5059578368469294


In [16]:
best_model_sub = None
best_result = 0 
best_depth = 0
for depth in range(1, 6): # seleccionaremos el rango del hiperparámetro
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345, class_weight = 'balanced')
    model.fit(features_downsampled, target_downsampled)
    predictions_valid = model.predict(features_valid) 
    result =  f1_score(target_valid, predictions_valid)
    if result > best_result:
        best_model_sub = model
        best_result = result
        best_depth = depth

print(f"f1 con subremuestreo del mejor modelo con el conjunto de validación (max_depth = {best_depth}): {best_result}")


f1 con subremuestreo del mejor modelo con el conjunto de validación (max_depth = 5): 0.5632730732635585


In [17]:
best_model_rf_sub = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range (1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight = 'balanced')
        model.fit(features_downsampled, target_downsampled)
        predictions_valid = model.predict(features_valid) # obténdremos las predicciones del modelo con el conjunto de validación
        result = f1_score(target_valid, predictions_valid) 
        if result > best_result:
            best_model_rf_sub = model
            best_result = result
            best_est = est
            best_depth = depth

print("f1 con subremuestreo del mejor modelo con el conjunto de validación", best_result, "n_estimators:", best_est, "best_depth:", depth)

f1 con subremuestreo del mejor modelo con el conjunto de validación 0.6078838174273858 n_estimators: 60 best_depth: 9


Lo que hizimos en este submuestreo fue dividir la cantidad de zeros que teniamos en nuestro conjunto de datos para que fuera una cantidad similar a la de los unos que teniamos para de esta forma poder ayudar a que nuestro modelo pudiera mejorar en la prediccion de zeros. Tanto el sobremuestreo como el submuestreo funcionan para hacer que la cantidad de zeros y unos en nuestro conjunto de datos en nuestro objetivo sean similares para que ayuden a nuestro modelos a tener mejores predicciones al estar mas equilibrado. Para poder saber que tan desquilibrados estaban y saber por cual numero teniamos que dividir solo imprimiamos `target_downsampled.value_counts()` y modificabamos el numero que representaba la fraccion para que los zeros disminuyeran y fueran una cantidad similar a los unos. Nuestro mejor modelo fue de igual manera el bosque aleatorio con una calificacion de `.60` superando nuestra meta pero al tener una mejor calificacion nuestro modelo anterior con la tecnica del sobremuestro nos quedaremos por lo tanto con nuestro modelo anterior.

# Examinaremos los valores AUC-ROC de nuestro mejor modelo

In [18]:
last_model = best_model_rf_sob
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8683477617407598


Nuestro valor AUC-ROC nos da un valor de `0.86` muy superior a 0.5 lo que significa que nuestro modelo distingue muy bien los valores positivos y negativmos, mejor que un clasificador aleatorio, por lo tanto nuestro modelo si funciona.

# Realizaremos la prueba final con nuestro mejor modelo

In [19]:
# utilizaremos nuestro mejor modelo de bosque aleatorio de regresion con el conjunto de prueba

predictions_test = last_model.predict(features_test) 
result =  f1_score(target_test, predictions_test)
print(result)

0.602905569007264


Nuestro modelo supero la prueba al tener una calificacion f1 de `.60` con el conjunto de prueba, lo cual es una muy buena señal.

# Conclusion general
En este proyecto utilizamos diferentes funciones que ayudaron a mejorar la calidad de los datos que ingresabamos a nuestros modelos, lo cual mejoro de forma significativa las predicciones de nuestro modelo. Me gustaria destacar 3 puntos principales de este proyecto.

1. Nuestro mejor modelo de este proyecto es `best_model_rf_sob` con una calificacion f1 de `0.64` con Sobremuestreo y una calificacion f1 con nuestro conjunto de prueba de `0.60`
2. El sobremuestreo y submuestreo son dos tecnicas que nos ayudan a poder equilibrar las clases.
3. Y los valores AUC-ROC nos ayudan a determinar si nuestro modelo clasifica mejor los valores positivos y negativos que un simple clasificador aleatorio al tener un valor superior a `0.5`.